In [1]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_plt

from pprint import pprint
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("data/stock_data.csv")
df.head()

,date,close,volume,open,high,low,symbol
0,2020-07-23,122.93,11010220,127.74,129.8494,121.32,SQ
1,2020-07-22,129.18,8437494,127.49,131.7100,127.06,SQ
2,2020-07-21,126.99,10787520,129.95,130.8000,125.07,SQ
3,2020-07-20,128.39,12423990,121.26,129.4600,120.53,SQ
4,2020-07-17,120.73,6968161,120.98,122.3800,118.59,SQ


In [3]:
# Get only the valid columns
df = df [['date', 'symbol', 'close']]

# Make symbol the headers (Pivot)
df = df.pivot(
    index='date',
    columns='symbol',
    values='close'
)

df.head()

symbol,AAPL,MSFT,SQ
date,,,
2020-01-24,318.31,165.04,71.14
2020-01-27,308.95,162.28,71.05
2020-01-28,317.69,165.46,74.80
2020-01-29,324.34,168.04,76.16
2020-01-30,323.87,172.78,76.10


In [4]:
assert df.shape[1] == 3

In [5]:
# Print Functionality
def print_func(title: str, df: pd.DataFrame):
    print('')
    print('='*80)
    print(title)
    print('-'*80)
    print(df)
    print('-'*80)

In [6]:
# Calculate Log returns
log_return = np.log(1 + df.pct_change())

# Generate Random Weights (of each portfolio)
random_weights = np.array(np.random.random(3))
random_weights /= np.sum(random_weights) # Summing to 1

# Calculate Expected Return (Rp) for the whole year, i.e. 252 working days
rp = np.sum((log_return.mean() * random_weights) * 252)

# Calculate Expected Volatility, for whole year
exp_vol = np.sqrt(
    np.dot(
        random_weights.T,
        np.dot(
            log_return.cov() * 252,
            random_weights
        )
    )
)

# Calculate Sharpe Ratio
sharpe_ratio = rp / exp_vol

weights_df = pd.DataFrame(data={
    'random_weights': random_weights,
})
print_func('PORTFOLIO WEIGHTS:', weights_df)

metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': rp,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])
print_func('PORTFOLIO METRICS:', metrics_df)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
   random_weights
0        0.793967
1        0.097030
2        0.109003
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  \
0                    0.407066                       0.549427   

   Portfolio Sharpe Ratio  
0                0.740891  
--------------------------------------------------------------------------------


### Monte Carlo Simulation

In [7]:
num_of_portfolios = 6000 # Number of simulations
np.random.seed(42)

all_weights = np.zeros((num_of_portfolios, 3)) # 5000 iter for 3 stocks
ret_arr = np.zeros(num_of_portfolios)
vol_arr = np.zeros(num_of_portfolios)
sharpe_arr = np.zeros(num_of_portfolios)

for i in range(num_of_portfolios):
    if i > 0 and i <= 1000:
        # First segment: Favor portfolio 1 (higher alpha for portfolio 1)
        alpha = [2.0, 0.5, 0.5]
    elif i > 1000 and i <= 2000:
        # Second segment: Favor portfolio 2
        alpha = [0.5, 2.0, 0.5]
    elif i > 2000 and i <= 3000:
        # Third segment: Favor portfolio 3
        alpha = [0.5, 0.5, 2.0]
    elif i > 3000 and i <= 4000:
        # Fourth segment: More balanced weights
        alpha = [1.0, 1.0, 1.0]
    elif i > 4000 and i <= 5000:
        # Fifth segment: Favor portfolios 2 and 3
        alpha = [0.5, 1.5, 1.5]
    else:
        # Sixth segment: Favours portfolios 1 and 3
        alpha = [1.5, 0.5, 1.5]
        
    # Using Dirichlet Distribution to get the random values and storing it
    all_weights[i] = np.random.dirichlet(alpha)
    
    # Calc expected log returns
    ret_arr[i] = np.sum((log_return.mean() * all_weights[i]) * 252)
    
    # Calc Volatility
    vol_arr[i] = np.sqrt(
        np.dot(all_weights[i].T, np.dot(log_return.cov() * 252, all_weights[i]))
    )
    
    # Calc Sharpe Ratio
    sharpe_arr[i] = ret_arr[i] / vol_arr[i]

In [8]:
# Create new Dataframe 
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]
simulations_df = pd.DataFrame(data=simulations_data).T

# Giving Column Names
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

simulations_df = simulations_df.infer_objects()

print_func('SIMULATIONS RESULT:', simulations_df.head())


SIMULATIONS RESULT:
--------------------------------------------------------------------------------
    Returns  Volatility  Sharpe Ratio  \
0  0.577228    0.605122      0.953904   
1  0.359591    0.534973      0.672167   
2  0.360827    0.541885      0.665874   
3  0.725873    0.677432      1.071507   
4  0.327959    0.533440      0.614799   

                                   Portfolio Weights  
0  [0.5618550811183951, 0.11678220993442089, 0.32...  
1  [0.5585504895960757, 0.43603329228225574, 0.00...  
2  [0.9112671646679144, 0.029423989689246692, 0.0...  
3  [0.40527230562006866, 0.08103788397330082, 0.5...  
4  [0.8437643822468637, 0.15454032565470158, 0.00...  
--------------------------------------------------------------------------------


### Getting important Metrics

In [9]:
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()] # Mazimum sharpe Ratio

min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()] # Minimum risk (volatility)

print_func('MAX SHARPE RATIO: ', max_sharpe_ratio)
print_func('MIN VOLATILITY: ', min_volatility)


MAX SHARPE RATIO: 
--------------------------------------------------------------------------------
Returns                                                       1.102282
Volatility                                                    0.916197
Sharpe Ratio                                                  1.203107
Portfolio Weights    [0.0002308664258650205, 0.00031463553034503017...
Name: 2565, dtype: object
--------------------------------------------------------------------------------

MIN VOLATILITY: 
--------------------------------------------------------------------------------
Returns                                                       0.334125
Volatility                                                    0.532886
Sharpe Ratio                                                  0.627012
Portfolio Weights    [0.7717725976852945, 0.2282273965002996, 5.814...
Name: 384, dtype: object
--------------------------------------------------------------------------------


In [10]:
min_volatility = simulations_df.loc[384]
min_volatility['Portfolio Weights']

array([7.71772598e-01, 2.28227397e-01, 5.81440592e-09])